In [ ]:
using Revise

In [ ]:
using Polyhedra
using CDDLib
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyDynamics
using RigidBodyDynamics: Bounds
using StaticArrays
using RigidBodyTreeInspector
using MechanismComplementarity
using JuMP, Gurobi

In [ ]:
function DrakeVisualizer.addgeometry!(vis::Visualizer, obs::Obstacle, boundary::HRepresentation=SimpleHRepresentation(vcat(eye(3), -eye(3)), vcat([1., 0.5, 2.0], -[-1., -0.5, -0.1])))
    p = intersect(boundary, obs.interior)
    addgeometry!(vis, CDDPolyhedron{3, Float64}(p))
end

In [ ]:
mechanism = parse_urdf(Float64, "box.urdf")
core = findbody(mechanism, "core")
floating_base = joint_to_parent(core, mechanism)
set_joint_type!(floating_base, Planar([1., 0, 0], [0., 0, 1.]))
floating_base.position_bounds = [Bounds(-5., 5), Bounds(0., 3), Bounds(-2π, 2π)]
floating_base.velocity_bounds = [Bounds(-10., 10), Bounds(-10., 10), Bounds(-2π, 2π)]
floating_base.effort_bounds .= Bounds(0., 0)

In [ ]:
world = root_body(mechanism)

basevis = Visualizer()[:world]
delete!(basevis)
vis = basevis[:planar]
setgeometry!(vis, mechanism, parse_urdf("box.urdf", mechanism))

floor = planar_obstacle(default_frame(world), [0, 0, 1.], [0, 0, 0.])
free_space = space_between([floor])

addgeometry!(basevis[:environment], floor)

core = findbody(mechanism, "core")

env = Environment(
    Dict(core => ContactEnvironment(
                [
                Point3D(default_frame(core), SVector(0.1, 0, 0.2)),
                Point3D(default_frame(core), SVector(-0.1, 0, 0.2)),
                Point3D(default_frame(core), SVector(0.1, 0, -0.2)),
                Point3D(default_frame(core), SVector(-0.1, 0, -0.2)),
                 ],
                [floor],
                [free_space])))

In [ ]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, floating_base, [-1, 0.3, -0.3])
configuration_derivative_to_velocity!(velocity(x0, floating_base), floating_base, configuration(x0, floating_base), [3., 0, 0])

settransform!(vis, x0)

controller = x -> begin
    zeros(num_velocities(x))
end

Δt = 0.05

results = MechanismComplementarity.simulate(x0, controller, env, Δt, 100, GurobiSolver(OutputFlag=0));

In [ ]:
for r in results
    set_configuration!(x0, configuration(r.state))
    settransform!(vis, x0)
    sleep(Δt)
end

In [ ]:
urdf_mech = parse_urdf(Float64, "box.urdf")
mechanism2, base2 = planar_revolute_base()
attach!(mechanism2, base2, urdf_mech)
world2 = root_body(mechanism2)

vis2 = basevis[:dummy]
setgeometry!(vis2, mechanism2, parse_urdf("box.urdf", mechanism2))

core2 = findbody(mechanism2, "core")
floor2 = planar_obstacle(default_frame(world2), [0, 0, 1.], [0, 0, 0.])
free_space2 = space_between([floor2])
env2 = Environment(
    Dict(core2 => ContactEnvironment(
                [
                Point3D(default_frame(core2), SVector(0.1, 0, 0.2)),
                Point3D(default_frame(core2), SVector(-0.1, 0, 0.2)),
                Point3D(default_frame(core2), SVector(0.1, 0, -0.2)),
                Point3D(default_frame(core2), SVector(-0.1, 0, -0.2)),
                 ],
                [floor2],
                [free_space2])))

x02 = MechanismState{Float64}(mechanism2)
set_velocity!(x02, zeros(num_velocities(x02)))
set_configuration!(x02, findjoint(mechanism2, "base_x"), [-1])
set_configuration!(x02, findjoint(mechanism2, "base_z"), [0.3])
set_configuration!(x02, findjoint(mechanism2, "base_rotation"), [0.3])
set_velocity!(x02, findjoint(mechanism2, "base_x"), [3])

settransform!(vis2, x02)

controller2 = x -> begin
    zeros(num_velocities(x))
end

Δt = 0.05

@time results2 = MechanismComplementarity.simulate(x02, controller2, env2, Δt, 100, GurobiSolver(OutputFlag=0));

In [ ]:
for (r1, r2) in zip(results, results2)
    set_configuration!(x0, configuration(r1.state))
    settransform!(vis, x0)
    set_configuration!(x02, configuration(r2.state))
    settransform!(vis2, x02)
    sleep(Δt)
end